In [1]:
import subprocess
import threading
import queue
import time
from scapy.all import sniff

# 버퍼 및 큐 초기화
log_buffer = queue.Queue()
network_buffer = queue.Queue()
write_buffer = queue.Queue()  # write buffer 추가
db = queue.Queue()  # DB 시뮬레이션

In [2]:
def fetch_windows_events():
    """PowerShell을 사용하여 윈도우 이벤트 로그 조회"""
    while True:
        command = 'Get-WinEvent -LogName System -MaxEvents 10 | Format-List'
        process = subprocess.Popen(["powershell", "-ExecutionPolicy", "Bypass", "-Command", command], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        result, error = process.communicate()

        if error:
            print(f"Error: {error.decode('cp949')}")
        else:
            output = result.decode('cp949')
            events = output.split('\r\n\r\n')
            for event in events:
                if event.strip():
                    print(f"네트워크 로그 : {event}")
                    log_buffer.put((event, time.time()))
        
        time.sleep(5)  # 폴링 간격

In [3]:
def packet_handler(packet):
    """네트워크 패킷 핸들러"""
    print(f"네트워크 패킷 : {packet.summary()}")
    network_buffer.put((packet.summary(), time.time()))

In [4]:
def buffer_to_write_buffer():
    """Log buffer와 Network buffer에서 Write buffer로 데이터 이동, 이동 알림 출력"""
    while True:
        if not log_buffer.empty():
            data = log_buffer.get()
            write_buffer.put(data)
            # print(f"Write Buffer로 이동 (Log): {data[0]}")  # 이동 알림 출력
        
        if not network_buffer.empty():
            data = network_buffer.get()
            write_buffer.put(data)
            # print(f"Write Buffer로 이동 (Network): {data[0]}")  # 이동 알림 출력
        
        time.sleep(0.5)

In [5]:
def db_writer():
    """Write buffer에서 DB로 데이터 이동 및 저장"""
    while True:
        if not write_buffer.empty():
            data, start_perf_time = write_buffer.get()
            start_time = time.perf_counter()  # 데이터 처리 시작 시간 (높은 정밀도)
            db.put(data)  # DB에 저장
            end_time = time.perf_counter()  # 데이터 처리 완료 시간
            processing_time = end_time - start_time  # 처리 시간 계산
            # print(f"DB 저장: {data}, 데이터당 처리 시간: {processing_time:.9f}초")
        
        time.sleep(0.5)  # DB에 쓰기 전 간단한 딜레이

In [ ]:
# 스레드 실행
threading.Thread(target=fetch_windows_events, daemon=True).start()
threading.Thread(target=lambda: sniff(prn=packet_handler, store=False), daemon=True).start()
threading.Thread(target=buffer_to_write_buffer, daemon=True).start()  # Log와 Network buffer에서 Write buffer로 이동 처리
threading.Thread(target=db_writer, daemon=True).start()

try:
    while True:
        time.sleep(1)
except KeyboardInterrupt:
    print("프로그램 종료")

네트워크 패킷 : Ether / IP / TCP 13.69.239.73:https > 192.168.0.217:54722 SA
네트워크 패킷 : Ether / IP / TCP 192.168.0.217:54722 > 13.69.239.73:https A
네트워크 패킷 : Ether / IP / TCP 192.168.0.217:54722 > 13.69.239.73:https PA / Raw
네트워크 패킷 : Ether / ARP who has 192.168.0.57 says 192.168.0.206 / Padding
네트워크 패킷 : Ether / IP / TCP 104.208.16.91:https > 192.168.0.217:54457 A / Padding
네트워크 패킷 : Ether / IP / UDP 172.30.142.83:55509 > 239.255.255.250:ws_discovery / Raw
네트워크 패킷 : Ether / IP / TCP 13.69.239.73:https > 192.168.0.217:54722 PA / Raw
네트워크 패킷 : Ether / IP / TCP 192.168.0.217:54722 > 13.69.239.73:https PA / Raw
네트워크 패킷 : Ether / IP / UDP 172.30.142.69:64137 > 239.255.255.250:ssdp / Raw
네트워크 패킷 : Ether / IP / UDP 172.30.142.110:57079 > 239.255.255.250:ssdp / Raw
네트워크 패킷 : Ether / IP / TCP 13.69.239.73:https > 192.168.0.217:54722 A / Raw
네트워크 패킷 : Ether / IP / TCP 13.69.239.73:https > 192.168.0.217:54722 A / Raw
네트워크 패킷 : Ether / IP / TCP 192.168.0.217:54722 > 13.69.239.73:https A
네트워크 패킷 : Ether 

네트워크 로그 : TimeCreated  : 2024-04-17 오후 12:00:00
ProviderName : EventLog
Id           : 6013
Message      : 시스템 작동 시간은 418741초입니다.
네트워크 로그 : TimeCreated  : 2024-04-17 오전 11:48:22
ProviderName : Microsoft-Windows-DistributedCOM
Id           : 10016
Message      : 응용 프로그램별 권한 설정에서 CLSID가 
               {2593F8B9-4EAF-457C-B68A-50F6B8EA6B54}
               이고 APPID가 
               {15C20B67-12E7-4BB6-92BB-7AFF07997402}
               인 COM 서버 응용 프로그램에 대한 로컬 활성화 사용 권한을 응용 프로그램 컨테이너 사용할 수 없음 SID(사용
               할 수 없음)에서 실행 중인 주소 LocalHost(LRPC 사용)의 사용자 DESKTOP-K7R5T4T\41259 SID(S-1-5-21-40379830
               92-3729845704-2333426984-1001)에게 부여하지 않았습니다. 구성 요소 서비스 관리 도구를 사용하여 이 보안 권
               한을 수정할 수 있습니다.
네트워크 로그 : TimeCreated  : 2024-04-17 오전 11:47:49
ProviderName : Microsoft-Windows-Time-Service
Id           : 158
Message      : 시간 공급자 'VMICTimeProvider'에서 현재 하드웨어 및 작동 환경이 지원되지 않아 중지되었다고 표시했습니다.
                이 동작은 HyperV 게스트가 아닌 환경의 VMICTimeProvider에 필요합니다. 현재 작동 환경의 현재 공급자에